This Notebook will train its own word emebeddings instead of using pr-trained word embeddings

<br>V1: Initial version with Embedding layer without pre-trained word emebeddings: Public Score:0.85842
<br>V2: Apply Text Cleaning by cleaning the puntuations and converting text to lower case:Public Score:0.86376
<br>V3: Chnage the max_words to 20K and Embedding Dimensions to 100 : Public Score 0.84811
<br>V5: Use CUDNNGRU Model Architecture:

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os,gc
print(os.listdir("../input"))



from keras.optimizers import RMSprop, Adam
from keras.models import Model, Sequential
from keras.layers import Flatten, Dense, Embedding, Dropout
from keras.callbacks import EarlyStopping
from keras import losses, metrics, optimizers
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

pd.options.display.max_colwidth = 500
# Any results you write to the current directory are saved as output.

['train.csv', 'sample_submission.csv', 'test.csv']


Using TensorFlow backend.


### Read Data

In [2]:
train = pd.read_csv('../input/train.csv')
test  = pd.read_csv('../input/test.csv')
print('Train Shape{} Test Shape{}'.format(train.shape, test.shape))
train.head()

Train Shape(1804874, 45) Test Shape(97320, 2)


,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,bisexual,black,buddhist,christian,female,heterosexual,hindu,homosexual_gay_or_lesbian,intellectual_or_learning_disability,jewish,latino,male,muslim,other_disability,other_gender,other_race_or_ethnicity,other_religion,other_sexual_orientation,physical_disability,psychiatric_or_mental_illness,transgender,white,created_date,publication_id,parent_id,article_id,rating,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count
0,59848,0.000000,"This is so cool. It's like, 'would you want your mother to read this??' Really great idea, well done!",0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:41.987077+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4
1,59849,0.000000,"Thank you!! This would make my life a lot less anxiety-inducing. Keep it up, and don't let anyone get in your way!",0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:42.870083+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4
2,59852,0.000000,This is such an urgent design problem; kudos to you for taking it on. Very impressive!,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:45.222647+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4
3,59855,0.000000,Is this something I'll be able to install on my site? When will you be releasing it?,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:47.601894+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4
4,59856,0.893617,haha you guys are a bunch of losers.,0.021277,0.0,0.021277,0.87234,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2015-09-29 10:50:48.488476+00,2,NaN,2006,rejected,0,0,0,1,0,0.0,4,47


# Clean Data
Adding preprocessing from this kernel: https://www.kaggle.com/taindow/simple-cudnngru-python-keras
> 

In [3]:
def clean_special_chars(text, punct, mapping):
        for p in mapping:
            text = text.replace(p, mapping[p])    
        for p in punct:
            text = text.replace(p, f' {p} ')     
        return text

def clean_text(df):
    df['comment_text'] = df['comment_text'].apply(lambda x: x.lower())
    punct_mapping = {"_":" ", "`":" "}
    punct = "/-'?!.,#$%\'()*+-/:;<=>@[\\]^_`{|}~" + '""“”’' + '∞θ÷α•à−β∅³π‘₹´°£€\×™√²—–&'
    df['comment_text'] = df['comment_text'].apply(lambda x: clean_special_chars(x, punct, punct_mapping))
    return df

In [4]:
train = clean_text(train)
test = clean_text(test)
train.head() 

,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,bisexual,black,buddhist,christian,female,heterosexual,hindu,homosexual_gay_or_lesbian,intellectual_or_learning_disability,jewish,latino,male,muslim,other_disability,other_gender,other_race_or_ethnicity,other_religion,other_sexual_orientation,physical_disability,psychiatric_or_mental_illness,transgender,white,created_date,publication_id,parent_id,article_id,rating,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count
0,59848,0.000000,"this is so cool . it ' s like , ' would you want your mother to read this ? ? ' really great idea , well done !",0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:41.987077+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4
1,59849,0.000000,"thank you ! ! this would make my life a lot less anxiety - inducing . keep it up , and don ' t let anyone get in your way !",0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:42.870083+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4
2,59852,0.000000,this is such an urgent design problem ; kudos to you for taking it on . very impressive !,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:45.222647+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4
3,59855,0.000000,is this something i ' ll be able to install on my site ? when will you be releasing it ?,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:47.601894+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4
4,59856,0.893617,haha you guys are a bunch of losers .,0.021277,0.0,0.021277,0.87234,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2015-09-29 10:50:48.488476+00,2,NaN,2006,rejected,0,0,0,1,0,0.0,4,47


In [5]:

max_words = 20000
maxlen = 220


tokenizer = Tokenizer(num_words = max_words, lower = True)
tokenizer.fit_on_texts(list(train['comment_text']) + list(test['comment_text']))
word_index = tokenizer.word_index

y = train.target.apply(lambda x: 0 if x < 0.5 else 1)

X = tokenizer.texts_to_sequences(list(train['comment_text']))
X_test =  tokenizer.texts_to_sequences(list(test['comment_text']) )  
                                       
X = pad_sequences(X, maxlen= maxlen)   
X_test =  pad_sequences(X_test, maxlen= maxlen)      



In [6]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.2,  random_state = 42,stratify = y )
del(train, test, X,y)
gc.collect()

57

## Define and Train Model

In [7]:
from keras.optimizers import RMSprop, Adam
from keras.layers import Embedding, Input, Dense, CuDNNGRU,concatenate, Bidirectional, SpatialDropout1D, Conv1D, GlobalAveragePooling1D, GlobalMaxPooling1D
#Model Architecture from https://www.kaggle.com/taindow/simple-cudnngru-python-keras
embedding_dim = 30

def get_model():
    sequence_input = Input(shape=(maxlen,), dtype='int32')
    embedding_layer = Embedding(len(tokenizer.word_index) + 1,
                                embedding_dim,                          
                                input_length=maxlen)

    x = embedding_layer(sequence_input)
    x = SpatialDropout1D(0.2)(x)
    x = Bidirectional(CuDNNGRU(64, return_sequences=True))(x)   
    x = Conv1D(64, kernel_size = 2, padding = "valid", kernel_initializer = "he_uniform")(x)

    avg_pool1 = GlobalAveragePooling1D()(x)
    max_pool1 = GlobalMaxPooling1D()(x)     

    x = concatenate([avg_pool1, max_pool1])

    preds = Dense(1, activation='sigmoid')(x)


    model = Model(sequence_input, preds)

    model.compile(loss='binary_crossentropy',
                  optimizer=Adam(),
                  metrics=['acc'])
    return model


In [8]:
model = get_model()
print(model.summary())
EPOCHS = 15
BATCH_SIZE = 512
print('Train Size{}, Validation Size {}, Test Size {}'.format(X_train.shape, X_valid.shape, X_test.shape))
print('Max Words {}, Max Length {}, Embedding Dimesions {}'.format(max_words, maxlen, embedding_dim))
history = model.fit( X_train,
                     y_train,
                    epochs = EPOCHS,
                    batch_size = BATCH_SIZE,
                    callbacks = [EarlyStopping(monitor = 'val_acc', patience = 3)],
                    validation_data = (X_valid, y_valid)
                   )

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Train Size(1443899, 220), Validation Size (360975, 220), Test Size (97320, 220)
Max Words 20000, Max Length 220, Embedding Dimesions 30
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 1443899 samples, validate on 360975 samples
Epoch 1/15
1443899/1443899 [==============================] - 151s 105us/step - loss: 0.1454 - acc: 0.9463 - val_loss: 0.1302 - val_acc: 0.9498
Epoch 2/15
1443899/1443899 [==============================] - 149s 103us/step - loss: 0.1264 - acc: 0.9513 - val_loss: 0.1280 - val_acc: 0.9509
Epoch 3/15
1443899/1443899 [==============================] - 149s 103us/step - loss: 0.1207 - acc: 0.9534 - val_loss: 0.1303 - val_acc: 0.9504
Epoch 4/15
1443899/1443899 [==============================] - 148s

In [9]:

y_pred = model.predict(X_valid)
print('Validation ROC AUC Score', roc_auc_score(y_valid, y_pred))

Validation ROC AUC Score 0.9494438974095838


 ### Predict On Best Epoch

In [10]:
history_dict = history.history
valid_acc = history_dict['val_acc'] 
best_epoch = valid_acc.index(max(valid_acc)) + 1
best_acc =  max(valid_acc)
print('Best Accuracy Score {}, is for epoch {}'.format( best_acc, best_epoch))

model = get_model()
history = model.fit( X_train,
                     y_train,
                    epochs = best_epoch,
                    batch_size = BATCH_SIZE,
                    validation_data = (X_valid, y_valid)
                   )

Best Accuracy Score 0.9509467414640903, is for epoch 2
Train on 1443899 samples, validate on 360975 samples
Epoch 1/2
1443899/1443899 [==============================] - 150s 104us/step - loss: 0.1457 - acc: 0.9463 - val_loss: 0.1299 - val_acc: 0.9503
Epoch 2/2
1443899/1443899 [==============================] - 149s 103us/step - loss: 0.1259 - acc: 0.9513 - val_loss: 0.1277 - val_acc: 0.9509


In [11]:

y_pred = model.predict(X_valid)
print('Validation ROC AUC Score', roc_auc_score(y_valid, y_pred))

Validation ROC AUC Score 0.953194612011901


### Predict On Test data

In [ ]:
y_pred = model.predict(X_test)
sub = pd.read_csv('../input/sample_submission.csv')
sub['prediction'] = y_pred
sub.to_csv('submission.csv', index = False)

sub.head()